In [18]:
import networkx as nx
import numpy as np
import cv2
import json
import networkx as nx
import re
import argparse
import glob
import os

from route_generator import iter_all_white_points, draw_waypoint, rdp_algorithm, recorded_find_path
from add_path_to_graph import add_path, read_adjlist_with_tuples

##################### Input Town number #####################

town = int(input("Enter town number: 1, 2, 3, or 10"))

if town == 10:
    waypoint_map = cv2.imread("./waypoint_maps/Town10HD.png", 1)
    town_name = "Town10HD"
else:
    town_name = f"Town0{town}"
    waypoint_map = cv2.imread(f"./waypoint_maps/{town_name}.png", 1)


##################### Read latest graph list of the given town #####################

list_of_files = glob.glob(f'graph_list/{town_name}/*') # * means all if need specific format then *.csv
latest_file = max(list_of_files, key=os.path.getctime)

if town == 10:
    index = int(latest_file[30:-8])
    new_file_name = latest_file[:30] + str(index+1) + latest_file[-8:]
else:
    index = int(latest_file[27:-8])
    new_file_name = latest_file[:27] + str(index+1) + latest_file[-8:]

G = read_adjlist_with_tuples(latest_file)
print(latest_file)

graph_list/Town02\2_graph_v10.adjlist


In [19]:
################# Change start, end point #################

reverse = input("Do reverse search or not? (y for reverse search, other for original search)\n")

start, end = (1807, 1044), (2339, 1039)

if reverse == "y":
    start, end = end, start


ValueError: invalid literal for int() with base 10: ''

In [17]:
if start == (0,0) or end == (0,0):
    print("Didn't change to your point!!!!!!!!")

all_whites_pos = iter_all_white_points(waypoint_map)

new_img = waypoint_map.copy()
new_img[start[0]][start[1]] = np.array([0, 0, 255])
new_img[end[0]][end[1]] = np.array([0, 0, 255])

cv2.circle(new_img, start[::-1], radius=5, color=(255, 0, 0), thickness=5)
cv2.circle(new_img, end[::-1], radius=5, color=(0, 255, 0), thickness=5)


print("Start point:", start)
print("End point:", end)
print("Finding Path...")
# final_path = find_path(start, end, all_whites_pos)
flag, final_path = recorded_find_path(start, end, all_whites_pos, "Forward", True)

##### Path not found #####
if flag == False:
    print("Not found!!!")
    print(final_path)
    for i, j in final_path[0]:
        new_img[i][j] = np.array([255, 255, 0])
    
    draw_img = draw_waypoint(waypoint_map, start, end, final_path[0])
    rdp_img, _ = rdp_algorithm(draw_img, final_path[0])
    cv2.imwrite(f"test/not_found/test__{start}_{end}.png", new_img)
else:
    print("found!!!")
    print(final_path[0])
    print("Adding to the graph list...")
    
    if reverse == "y":
        fp = final_path[0][::-1]
        fp = [fp]
        add_path(G, fp, new_file_name)
    else:
        add_path(G, final_path, new_file_name)
    
    print(new_file_name, "Store!")
    draw_img = draw_waypoint(waypoint_map, start, end, final_path[0])
    rdp_img, _ = rdp_algorithm(draw_img, final_path[0])
    cv2.imwrite(f"test/found/test__{start}_{end}.png", rdp_img)

Start point: (2339, 1039)
End point: (1807, 1044)
Finding Path...
Get!!!
found!!!
[(2339, 1039), (2335, 1039), (2331, 1039), (2327, 1039), (2323, 1039), (2319, 1039), (2315, 1039), (2311, 1039), (2307, 1039), (2303, 1039), (2299, 1039), (2295, 1039), (2291, 1039), (2287, 1039), (2283, 1039), (2279, 1039), (2275, 1039), (2271, 1039), (2267, 1039), (2263, 1039), (2259, 1039), (2255, 1039), (2251, 1039), (2247, 1039), (2243, 1039), (2239, 1039), (2235, 1039), (2231, 1039), (2227, 1039), (2223, 1039), (2219, 1039), (2215, 1039), (2211, 1039), (2207, 1039), (2203, 1039), (2199, 1039), (2195, 1039), (2191, 1039), (2187, 1039), (2183, 1039), (2179, 1039), (2175, 1039), (2171, 1039), (2167, 1039), (2163, 1039), (2159, 1039), (2155, 1039), (2151, 1039), (2147, 1039), (2143, 1039), (2139, 1039), (2135, 1039), (2131, 1039), (2127, 1039), (2123, 1039), (2119, 1039), (2115, 1039), (2111, 1039), (2107, 1039), (2103, 1039), (2099, 1039), (2095, 1039), (2091, 1039), (2087, 1039), (2083, 1039), (2079, 

In [16]:
############### Quick test for other start point with the same goal point
start, end = (1900, 2430), (2429, 2517)

fp = nx.shortest_path(G,  start,  end)
print(fp)

[(1900, 2430), (1901, 2434), (1901, 2466), (1910, 2493), (2017, 2517), (2125, 2517), (2233, 2517), (2341, 2517), (2429, 2517)]
